In [5]:
# pip install elsapy
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json
import pandas as pd
import numpy as np
import warnings

## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

## Initialize client
client = ElsClient(config['apikey'])

## Search box
search = input("Enter your keywords which you are searching for: ")
year = input("Please enter your year filter exp1= '< 2020' , exp2= '> 1995' exp3 =  '= 2020'")
fsearch = search + " AND PUBYEAR " + year
print("Your keywords: \"" + search + "\" " + "| Publish Year Criteria is " + year + " | Preparing your query!")

## Initialize doc search object using Scopus and execute search, retrieving 
#   all results
doc_srch = ElsSearch(fsearch,'scopus')
doc_srch.execute(client, get_all = False)
print ("doc_srch has", len(doc_srch.results), "results.")

# selecting the search result
filename = "dump.json"
with open(filename) as raw_file:
       # print(raw_file)
        json_input = json.load(raw_file)
        
# creating dataframe from json file      
df1 = pd.DataFrame.from_dict(json_input, orient='columns')

# Preprocessing giving a warning this one is silent it.
pd.set_option('mode.chained_assignment', None)

# taking the valid link from column
for i in range(len(df1.link)):
    df1.link[i] = df1.link[i][2]['@href']

# fixing affiliation mess    
for i in range(len(df1.affiliation)):
    if isinstance(df1.affiliation[i], float):
        df1.affiliation[i] = [{'@_fa': 'true','affilname': 'NaN','affiliation-city': 'NaN','affiliation-country': 'NaN'}]

df1['Inst'] = ''  
df1['Country'] = ''
df1['City'] = ''

for i in range(len(df1.affiliation)):
    df1['Inst'][i] = df1.affiliation[i][0]['affilname']
    df1['Country'][i] = df1.affiliation[i][0]['affiliation-country']
    df1['City'][i] = df1.affiliation[i][0]['affiliation-city']
# selecting neccessary features
df1 = df1[['citedby-count','dc:creator','prism:coverDisplayDate','dc:title','prism:publicationName','Inst','Country','City','prism:aggregationType','subtypeDescription','openaccess','prism:doi','link']]
#renaming features
df1= df1.rename(columns={'citedby-count':'Cited','dc:creator':'Author','prism:coverDisplayDate':'Date','dc:title':'Title','prism:publicationName':'Publication','prism:aggregationType':'Aggregation','subtypeDescription':'Subtype','openaccess':'OpenAccess','prism:doi':'Doi','link':'Link'})
df1.Cited= pd.to_numeric(df1.Cited)
df1 = df1.sort_values(["Cited","Date"],ascending= False,ignore_index= True)
df1.to_excel("result.xlsx")
print("The result has been saved into directory named as result.xlsx")
df1

Enter your keywords which you are searching for:  stock price prediction with deep learning
Please enter your year filter exp1= '< 2020' , exp2= '> 1995' exp3 =  '= 2020' < 2021


Your keywords: "stock price prediction with deep learning" | Publish Year Criteria is < 2021 | Preparing your query!
doc_srch has 25 results.
The result has been saved into directory named as result.xlsx


,Cited,Author,Date,Title,Publication,Inst,Country,City,Aggregation,Subtype,OpenAccess,Doi,Link
0,24,Coley C.W.,14 December 2020,Autonomous Discovery in the Chemical Sciences ...,Angewandte Chemie - International Edition,Massachusetts Institute of Technology,United States,Cambridge,Journal,Review,0,10.1002/anie.201909987,https://www.scopus.com/inward/record.uri?partn...
1,2,Gao W.,15 December 2020,Analysis on block chain financial transaction ...,Journal of Computational and Applied Mathematics,Changchun University of Science and Technology,China,Changchun,Journal,Article,0,10.1016/j.cam.2020.112991,https://www.scopus.com/inward/record.uri?partn...
2,2,Singh Chauhan G.,15 December 2020,A two-step hybrid unsupervised model with atte...,Expert Systems with Applications,Malaviya National Institute of Technology Jaipur,India,Jaipur,Journal,Article,0,10.1016/j.eswa.2020.113673,https://www.scopus.com/inward/record.uri?partn...
3,2,Calders K.,15 December 2020,Terrestrial laser scanning in forest ecology: ...,Remote Sensing of Environment,Universiteit Gent,Belgium,Ghent,Journal,Review,1,10.1016/j.rse.2020.112102,https://www.scopus.com/inward/record.uri?partn...
4,1,Li X.,16 December 2020,AI-based competition of autonomous vehicle fle...,European Journal of Operational Research,"University of Michigan, Ann Arbor",United States,Ann Arbor,Journal,Article,0,10.1016/j.ejor.2020.05.020,https://www.scopus.com/inward/record.uri?partn...
5,1,Lee S.W.,15 December 2020,Stock market forecasting with super-high dimen...,Expert Systems with Applications,Ajou University,South Korea,Kyonggi-do,Journal,Article,0,10.1016/j.eswa.2020.113704,https://www.scopus.com/inward/record.uri?partn...
6,1,D'Urso P.,15 December 2020,Cepstral-based clustering of financial time se...,Expert Systems with Applications,Università degli Studi di Roma La Sapienza,Italy,Rome,Journal,Article,0,10.1016/j.eswa.2020.113705,https://www.scopus.com/inward/record.uri?partn...
7,1,Woodley S.M.,11 December 2020,"Structure prediction of crystals, surfaces and...",Philosophical Transactions of the Royal Societ...,University College London,United Kingdom,London,Journal,Review,0,10.1098/rsta.2019.0600rsta20190600,https://www.scopus.com/inward/record.uri?partn...
8,1,Friedlingstein P.,11 December 2020,Global Carbon Budget 2020,Earth System Science Data,University of Exeter,United Kingdom,Exeter,Journal,Article,1,10.5194/essd-12-3269-2020,https://www.scopus.com/inward/record.uri?partn...
9,0,Wang F.,7 December 2020,Predicting S&amp;P 500 Market Price by Deep Ne...,E3S Web of Conferences,GREENVILLE,United States,Greenville,Conference Proceeding,Conference Paper,1,10.1051/e3sconf/202021402040,https://www.scopus.com/inward/record.uri?partn...
